<a target="_blank" href="https://colab.research.google.com/github/sarthakrastogi/quality-prompts/blob/main/examples/few_shot_cot_usage.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%%capture
!pip install quality-prompts

In [2]:
import requests
import json
import os
import copy

from quality_prompts.prompt import QualityPrompt
from quality_prompts.exemplars import ExemplarStore, Exemplar
from quality_prompts.utils.llm import get_embedding

In [3]:
os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY_HERE"

### Initialise sample exemplars for use in few-shot prompt

In [4]:
url = "https://github.com/sarthakrastogi/quality-prompts/raw/main/examples/math_science_problems_sample_exemplars.json"
response = requests.get(url)
math_science_problems_sample_exemplars = response.json()

exemplars = [Exemplar(input=e['input'], label=str(e['label']), input_embedding=get_embedding(e['input']), complexity_level=e['complexity_level']) for e in math_science_problems_sample_exemplars]
exemplar_store = ExemplarStore(exemplars=exemplars)

### Define your prompt's components and initialise it

In [5]:
directive = """Solve the given problem step by step."""

additional_information = ""

output_formatting = ""

# Creating three separate prompts for this example
prompt1 = QualityPrompt(
                        directive=directive,
                        additional_information=additional_information,
                        output_formatting=output_formatting,
                        exemplar_store=exemplar_store
                       )

prompt2 = copy.deepcopy(prompt1)
prompt3 = copy.deepcopy(prompt1)

In [6]:
phys_problem = "A car accelerates from rest at a constant rate of \( 3 \, \text{m/s}^2 \). Calculate the distance it covers in 10 seconds."

chem_problem = "Calculate the pH of a 0.05 M solution of acetic acid (CH3COOH), given that the \( K_a \) of acetic acid is \( 1.8 \times 10^{-5} \)."

### Contrastive CoT prompting

Adds exemplars with both correct and incorrect thoughts to show both how to and how not to think.

https://arxiv.org/pdf/2311.09277

In [7]:
prompt1.contrastive_cot_prompting(input_text=chem_problem)
print(prompt1.compile())

Solve the given problem step by step.
You are given examples of both valid and invalid reasoning for solving the problem. Observe these examples to understand how to and how not to reason about the problem.

Example input: Determine the pH of a 0.01 M HCl solution.
Example output: Valid Example:
Given that HCl is a strong acid that dissociates completely in water, we can determine the concentration of hydrogen ions \( [H^+] \) in the solution to be 0.01 M.
To find the pH of the solution, we use the formula \( \text{pH} = -\log[H^+] \).
Substitute the value of \( [H^+] \):
\( \text{pH} = -\log(0.01) = -\log(10^{-2}) = 2 \)
Therefore, the pH of the 0.01 M HCl solution is 2.

Invalid Example:
Since HCl is a strong acid, it means that the concentration of the resulting hydrogen ions is 1 M.
Calculating the pH using the formula \( \text{pH} = -\log[H^+] \), we get:
\( \text{pH} = -\log(1) = -0 = 0 \)
Therefore, the pH of the 0.01 M HCl solution is 0.
This is incorrect because the concentrat

### Uncertainty routed CoT
Samples multiple CoT reasoning paths, then selects the majority if it is above a certain threshold (calculated based on validation data). If not, it samples greedily and selects that response

https://storage.googleapis.com/deepmind-media/gemini/gemini_1_report.pdf

In [8]:
prompt2.uncertainty_routed_cot_prompting(
                                        input_text=chem_problem,
                                        n_reasoning_paths=4,
                                        temperature=0.4
                                        )
print(prompt2.compile())

Solve the given problem step by step.

Example input: Calculate the pH of a 0.05 M solution of acetic acid (CH3COOH), given that the \( K_a \) of acetic acid is \( 1.8 	imes 10^{-5} \).
Example output: After analyzing all the reasoning paths provided, it is clear that the majority of paths (3 out of 4 paths) follow the same logical steps to determine the pH of a 0.05 M solution of acetic acid. The winning reasoning path, which is used by the majority of LLMs, is Reasoning path 3. 

In Reasoning path 3:
1. The path starts by writing the dissociation equation of acetic acid and the equilibrium expression for the dissociation reaction.
2. The assumption that the concentrations of \(CH_3COO^-\) and \(H^+\) at equilibrium are considered to be equal leads to the setup of the equilibrium expression with the given \(K_a\) value.
3. By simplifying the equation based on the assumption that \(x\) is much smaller than 0.05, the path calculates the concentration of \(H^+\) ions (\(x\)) at equilibri

### Complexity based prompting

First searches the most complex exemplars for use in context.
Then samples multiple CoT reasoning paths, then selects the majority if it is above a certain threshold (calculated based on validation data). If not, it samples greedily and selects that response

https://openreview.net/pdf?id=yf1icZHC-l9
        

In [9]:
prompt3.complexity_based_prompting(
                                    input_text=chem_problem,
                                    n_reasoning_paths=4,
                                    temperature=0.4,
                                    n_exemplars=3
                                )
print(prompt3.compile())

Solve the given problem step by step.

Example input: Calculate the pH of a 0.05 M solution of acetic acid (CH3COOH), given that the \( K_a \) of acetic acid is \( 1.8 	imes 10^{-5} \).
Example output: Based on the reasoning paths provided, majority voting indicates that Reasoning path 2 is the most commonly used approach in solving the problem step by step. We will now explain the winning reasoning path in detail:

### Winning Reasoning Path (Reasoning path 2):
**1. Concept Introduction:**
- The reasoning path begins by introducing the concept of weak acid dissociation and the equilibrium constant expression for the dissociation reaction.

**2. Equilibrium Reaction:**
- It correctly writes the equilibrium reaction for the dissociation of acetic acid and presents the equilibrium constant expression.

**3. Equilibrium Concentrations at Equilibrium:**
- It correctly determines the concentrations of acetic acid, acetate ions, and hydrogen ions at equilibrium based on the assumption that t